In [ ]:
import matplotlib.patches as mpatches
import numpy as np
import os
import math
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
params = {
   'axes.labelsize': 12,
   'font.size': 10,
   'legend.fontsize': 11,
   'xtick.labelsize': 10,
   'ytick.labelsize': 12,
   'text.usetex': False,
   'figure.figsize': [10, 2] # instead of 4.5, 4.5
   }
plt.rcParams.update(params)

## Fixed Parameter

In [ ]:
# experiment
exp = 'dist_subj'
exp_label = 'Adding Independent Subjects'
roi_all = ['dmn']
model = 'multi_srm'

# dictionaries
cl_dict = {0:1/450,1:1/297,2:1/297,3:1/1973} # chance level of each dataset
feat_dict = {'multi_srm':[75,75,100],'multi_dict':[25,50,50],'avg':[50,50,50]}
roi_dict = {'dmn':0,'pt':1,'eac':2}
# change multi_srm name!
md_dict = {'avg':'MNI','multi_srm':'MDSRM','multi_dict':'MDDict'}
ds_dict = {0:'GreenEyes',1:'Milky',2:'Vodka',3:'Sherlock'}

# paths
input_file = '../../output/accu_bar/{}/{}/{}_ds{}_shared{}.npz' #exp,roi,model,ds,shared
output_path = '../../output/figures/{}/' #exp
output_file = output_path+'{}_{}_{}' #model,roi,ds
if not os.path.exists(output_path.format(exp)):
    os.makedirs(output_path.format(exp))

## Plotting Parameter

In [ ]:
ds = [1,3]
shared = [1]
min_accu = 0.115
max_accu = 0.137

## Aggregate Accuracy

In [ ]:
# Accuracies
all_mean = [] # length # of roi
all_se = [] # length # of roi
# aggregate each roi
for roi in roi_all:
    roi_mean = []
    roi_se = []
    # aggregate each number of shared subjects
    for sd in shared:
        ws=np.load(input_file.format(exp,roi,model,ds,sd).replace(' ',''))
        roi_mean.append(ws['mean'])
        roi_se.append(ws['se'])
    all_mean.append(roi_mean)
    all_se.append(roi_se)

## Plot all roi (line plot)

In [ ]:
color = 'red'
for m,roi in enumerate(roi_all):
    num_subj = all_mean[0][0].shape[0]
    idx = np.array(list(range(num_subj)),dtype=np.int32)
    plt.figure()
#     color_list = ['lightcoral','red','darkred','lightgreen','green','darkgreen','lightblue','blue','darkblue']
#     marker_list = ['x','x','x','o','o','o','+','+','+']
    lines = []
    for r,sd in enumerate(shared):
        mean = all_mean[m][r]
        se = all_se[m][r]        
        line = plt.errorbar (idx,mean,se, color=color,capsize=3, ecolor='k',linewidth=2.0) 
#         line = plt.errorbar (idx,mean,se, color=color_list[r],marker = marker_list[r],capsize=3, ecolor='k',linewidth=2.0,label=sd) 
        lines.append(line)
        
    left_lim = -0.5
    right_lim = num_subj-0.5
    plt.xlim([left_lim,right_lim])
    plt.ylabel('Accuracy')
    plt.xlabel('Number of independent subjects')
    plt.ylim([min_accu,max_accu])
    plt.xticks(idx,idx+1)

    # plot chance accuracy
    cl = cl_dict[ds[0]]
    line = plt.plot([left_lim, right_lim], [cl, cl], 'k-.', linewidth=2)

    

    # Add texts
    plt.text((right_lim+left_lim)/2, 1.01*(max_accu-min_accu)+min_accu, exp_label+', ROI: '+roi.upper(),fontsize=12,horizontalalignment='center', verticalalignment='bottom')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    plt.text((right_lim+left_lim)/2, 0.95*(max_accu-min_accu)+min_accu, 'k = '+str(feat_dict[model][roi_dict[roi]]),bbox=props,fontsize=12,horizontalalignment='right', verticalalignment='top')
    props2 = dict(boxstyle='square', facecolor='ivory',alpha=0.5)
    ds_text = 'Test:'+ds_dict[ds[0]]+'\nAdd:'+ds_dict[ds[1]]
    sd_text = '# Shared Subjects: '+str(shared[0])
    plt.text(left_lim+0.01*num_subj,0.95*(max_accu-min_accu)+min_accu,ds_text,bbox=props2,fontsize=12,horizontalalignment='left', verticalalignment='top')
    plt.text(left_lim+0.99*(right_lim-left_lim),0.95*(max_accu-min_accu)+min_accu,sd_text,bbox=props2,fontsize=12,horizontalalignment='right', verticalalignment='top')


#     plt.savefig(output_file.format(exp,roi,model,ds).replace(' ','')+'.eps', format='eps', dpi=200,bbox_inches='tight')
    plt.savefig(output_file.format(exp,roi,model,ds).replace(' ','')+'.pdf', format='pdf', dpi=200,bbox_inches='tight')